In [17]:
# 回到根目錄
%cd /content

!pip install -q openai faiss-cpu numpy

/content
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 53.5 MB/s eta 0:00:00


In [18]:
from google.colab import files

uploaded = files.upload()  # select the original material file

Saving NovaTech_Product_Intro_Bilingual.txt to NovaTech_Product_Intro_Bilingual.txt


In [19]:
filename = "NovaTech_Product_Intro_Bilingual.txt"  # your file name

with open(filename, "r", encoding="utf-8") as f:
    full_text = f.read()

# using a paragraph to chunk
text_chunks = [chunk.strip() for chunk in full_text.split("\n") if chunk.strip()]
print(f"split to {len(text_chunks)} chunks")

split to 56 chunks


In [20]:
from google.colab import userdata
from openai import OpenAI
import os

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

# embedding the texts
def embed_texts(texts):
    response = client.embeddings.create(
        input=texts,
        model="text-embedding-3-small"
    )
    return [item.embedding for item in response.data]

In [21]:
import faiss
import numpy as np

# generate embedding texts
embeddings = embed_texts(text_chunks)
embedding_dim = len(embeddings[0])

# create FAISS index
index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(embeddings).astype("float32"))

# store FAISS index
faiss.write_index(index, "docs_index.faiss")

# store chunks
with open("docs_chunks.txt", "w", encoding="utf-8") as f:
    for line in text_chunks:
        f.write(line + "\n")

print("✅ index and chunks stored")

✅ index and chunks stored


In [22]:
import faiss
import numpy as np

# read the index and the original paragraphs
index = faiss.read_index("docs_index.faiss")

with open("docs_chunks.txt", "r", encoding="utf-8") as f:
    chunks = [line.strip() for line in f if line.strip()]

In [23]:
def search_similar_chunks(query, k=3):
    # convert query to embedding vector
    embedding = embed_texts([query])[0]
    embedding = np.array(embedding).astype("float32").reshape(1, -1)

    # find out the nearest k embedding values
    distances, indices = index.search(embedding, k)

    print("🔍 query results：")
    for rank, idx in enumerate(indices[0]):
        print(f"\n[Top {rank+1}]")
        print(chunks[idx])

In [24]:
search_similar_chunks("哪個產品可以用來做邊緣運算？")

🔍 query results：

[Top 1]
AeroNode 是具備邊緣運算能力的工業 IoT 閘道器，支援多種通訊協定與本地事件判斷。

[Top 2]
公司與產品介紹內容（中英雙語）

[Top 3]
ChainGuard 透過區塊鏈技術提供文件與交易紀錄的不可篡改驗證，可應用於合約管理與供應鏈紀錄。
